In [7]:
!pip install --upgrade pip -q
!pip install --upgrade kaggle


In [8]:
!pip install numpy==1.23.0

#Запускаем

In [9]:
!pip install pydantic==1.8.1

In [11]:

!pip install -U openai -q
!pip install pytelegrambotapi -q
!pip install -U deep-translator -q
!pip install langchain tiktoken -q
!pip install pydub -q
#!pip install git+https://github.com/openai/whisper.git -q
!sudo apt install ffmpeg

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
chex 0.1.85 requires numpy>=1.24.1, but you have numpy 1.23.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have dis

In [12]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers
!pip install fpdf


In [16]:
from openai import OpenAI
import telebot
import matplotlib.pyplot as plt
from deep_translator import GoogleTranslator
from PIL import Image
import io
import base64
from langchain.chat_models import ChatOpenAI
import os

In [8]:
#Запускаем ниже остальное

In [17]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

Нужно скачать файл модели вот отсюда https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/tree/main , подойдет любая с форматом guff/ но чем больше размер тем дольше обработка. В ячейки ниже при параметрах  n_ctx=4096 и

```
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
text_chunks = text_splitter.split_documents(data)
```


 средняя время обработки, например  уголовного кодекса 4-7 минут(300 страниц) одного вопроса, но маленькие 1-5 страниц (пол минуты - 1.5минуты), но любой первый вопрос он будет обрабатывать медленее чем обычно

1.   А при n_ctx=4096, chunk_size=10000, chunk_overlap=2000 больше часа. Я остановил спустя 1:12/ И не знаю за сколько он справится. Как интересный пример, оставлю на ночь и посмотрю. Но это уже не в проекте.



In [18]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/kaggle/input/mistral4gb/mistral-7b-instruct-v0.1.Q4_K_S.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=8096,

)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /kaggle/input/mistral4gb/mistral-7b-instruct-v0.1.Q4_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:  

In [19]:
print(llm)

LlamaCpp
Params: {'model_path': '/kaggle/input/mistral4gb/mistral-7b-instruct-v0.1.Q4_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.75, 'top_p': 1.0, 'logprobs': None, 'echo': False, 'stop_sequences': [], 'repeat_penalty': 1.1, 'top_k': 40}


#Установка библиотек для распознования голосов

In [20]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

from fpdf import FPDF
!pip install reportlab

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph

# Регистрация пользовательского шрифта
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 13.3 MB/s eta 0:00:00


#Распознаем голоса и составляем диалог в формате txt и его превращаем в pdf и сохраняем в Data так же нужно установить шрифт, для корректного отображеня PDF файла. Готовый PDF будет на гит хабе, чтобы не занимать память графического процессора в колабе,можно этот шаг пропустить предварительно загрузив в data PDF файлы, который нам понадобиться при анализе текста, в гит хабе будут готовый обработанный диалог в PDF и один большой PDF файл, кодекс РФ

##Дальше снизу идет код по обработке аудиофайла

# model_size: в зависимости от выбранной вами версии 'tiny', 'base', 'small', 'medium', 'large', мы получим разное качество распознавания

In [21]:
num_speakers = 2 #@param {type:"integer"}

language = 'any' #@param ['any', 'English']

model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.88G/2.88G [00:45<00:00, 67.8MiB/s]


#Загружаем аудио и обрабатываем
можем этот шаг пропустить,и загрузить вручную. Так памяти будет больше для анализа

In [26]:
pip install dejavu-fonts-ttf

ERROR: Could not find a version that satisfies the requirement dejavu-fonts-ttf (from versions: none)
ERROR: No matching distribution found for dejavu-fonts-ttf
Note: you may need to restart the kernel to use updated packages.


In [27]:
# upload audio file

path = '/kaggle/input/testwav/AUD-20231127-WA0003'
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

result = model.transcribe(path)
segments = result["segments"]
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

embeddings1 = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings1[i] = segment_embedding(segment)

embeddings1 = np.nan_to_num(embeddings1)

clustering = AgglomerativeClustering(num_speakers).fit(embeddings1)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()



##КОнвертируем в PDF##################################################

#Установи шриф(в гит хабе естғ фаЙл)
font_path = '/kaggle/input/dejfont/DejaVuSans.ttf'
pdfmetrics.registerFont(TTFont("DejaVu", font_path))

# Создание стилей для абзацев
styles = getSampleStyleSheet()
custom_style = ParagraphStyle(
    'CustomStyle',
    parent=styles['Normal'],
    fontName='DejaVu',
    fontSize=12,
)

def create_pdf(input_file, output_file):
    # Создание PDF-документа
    pdf = SimpleDocTemplate(output_file, pagesize=letter)

    # Открытие текстового файла в режиме чтения
    with open(input_file, "r", encoding='utf-8') as file:
        # Сборка Paragraphs для вставки в документ
        paragraphs = []
        for line in file:
            paragraphs.append(Paragraph(line, custom_style))

    # Добавление Paragraphs в документ
    pdf.build(paragraphs)

# Конвертация текстового файла в PDF
input_file = "transcript.txt"
output_file = "/kaggle/working/transcript.pdf"
create_pdf(input_file, output_file)



ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

#Отсөда начинает Анализ текста с PDF фалов, которые лежат в папке Data :/content/sample_data/Data

#Работ по тексту, все файлы PDF в папке Data(их может быть несколько) обрабатываются и по этим файлам бот находит ответы.

In [71]:
loader = PyPDFDirectoryLoader("/kaggle/input/fakedata/")
data = loader.load()

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
text_chunks = text_splitter.split_documents(data)
len(text_chunks)
text_chunks[0]

vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#тут без цикла,вводим запрос в переменную query

In [72]:


qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [33]:
query = "О чём говорится в разговоре? "

In [35]:
promt_template = "Ты мультилингвальный дружелюбный помощник, который всё знает. Выполни, что тебя просят. Если тебя попросят ответ, но ты его не знаешь, то ответ что не знаешь."

In [36]:
import tempfile
import os
from pydub import AudioSegment

bot = telebot.TeleBot('6507009266:AAGOY56MPIsTP7qOj8rJcAKT1a1m1WID4WI')#api telegram bot

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, 'Привет! Можешь спрашивать меня! Что тебя интересует?')

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    user_input =  message.text # Добавляем сообщение в промпт
    bot.send_message(message.from_user.id, "Я обрабатываю ваш запрос. Ожидайте 2-5 минут")
    result = qa.run(user_input) # Ответ модели
    #prediction = GoogleTranslator(source='auto', target='ru').translate(getResponse_(message.text))
    #message = getResponse(message.text)
    bot.send_message(message.from_user.id, result)

bot.polling(none_stop=True, interval=0)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1762.07 ms
llama_print_timings:      sample time =      15.68 ms /    29 runs   (    0.54 ms per token,  1849.73 tokens per second)
llama_print_timings: prompt eval time =    3015.66 ms /    14 tokens (  215.40 ms per token,     4.64 tokens per second)
llama_print_timings:        eval time =    8398.43 ms /    28 runs   (  299.94 ms per token,     3.33 tokens per second)
llama_print_timings:       total time =   11555.36 ms /    42 tokens


In [40]:
pip install python-telegram-bot

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.6/552.6 kB 8.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.26.0
    Uninstalling httpx-0.26.0:
      Successfully uninstalled httpx-0.26.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tempfile
import os
import subprocess
from pydub import AudioSegment
import subprocess
import contextlib
import wave
import numpy as np
from pydub import AudioSegment
from sklearn.cluster import AgglomerativeClustering
import datetime
from reportlab.lib.pagesizes import letter

from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

bot = telebot.TeleBot('6507009266:AAGOY56MPIsTP7qOj8rJcAKT1a1m1WID4WI')  # ваш API-ключ для Telegram Bot

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, 'Привет! Можешь спрашивать меня! Что тебя интересует?')

@bot.message_handler(content_types=['text', 'audio'])
def handle_messages(message):
    if message.content_type == 'text':
        user_input = message.text
        bot.send_message(message.from_user.id, "Я обрабатываю ваш запрос. Ожидайте 2-5 минут")
        loader = PyPDFDirectoryLoader("/kaggle/working/")
        data = loader.load()

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
        text_chunks = text_splitter.split_documents(data)

        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
        qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))
        result = qa.run(user_input)
        bot.send_message(message.from_user.id, result)
    elif message.content_type == 'audio':
        # Сохраняем аудиофайл, чтобы обработать его
        file_id = message.audio.file_id
        file_info = bot.get_file(file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        temp_file_path = "/kaggle/working/audio_input.ogg"  # Выберите формат, поддерживаемый pydub
        with open(temp_file_path, 'wb') as new_file:
            new_file.write(downloaded_file)

        # Обрабатываем аудиофайл
        result = process_audio(temp_file_path)
        bot.send_message(message.from_user.id, result)


# Функция для обработки аудиофайла
def process_audio(audio_path):
    if audio_path[-3:] != 'wav':
        subprocess.call(['ffmpeg', '-i', audio_path, 'audio.wav', '-y'])
        audio_path = 'audio.wav'

    result = model.transcribe(audio_path)
    segments = result["segments"]

    with contextlib.closing(wave.open(audio_path, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)

    audio = Audio()

    def segment_embedding(segment):
        start = segment["start"]
        end = min(duration, segment["end"])
        clip = Segment(start, end)
        waveform, sample_rate = audio.crop(audio_path, clip)
        return embedding_model(waveform[None])

    embeddings1 = np.zeros(shape=(len(segments), 192))
    for i, segment in enumerate(segments):
        embeddings1[i] = segment_embedding(segment)

    embeddings1 = np.nan_to_num(embeddings1)

    clustering = AgglomerativeClustering(num_speakers).fit(embeddings1)
    labels = clustering.labels_
    for i in range(len(segments)):
        segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

    def time(secs):
        return datetime.timedelta(seconds=round(secs))

    transcript_content = ""
    for (i, segment) in enumerate(segments):
        if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
            transcript_content += "\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n'
        transcript_content += segment["text"][1:] + ' '

    transcript_file_path = "/kaggle/working/transcript.txt"
    with open(transcript_file_path, "w") as f:
        f.write(transcript_content)

    # Конвертация текстового файла в PDF
    font_path = '/kaggle/input/dejfont/DejaVuSans.ttf'
    pdfmetrics.registerFont(TTFont("DejaVu", font_path))

    styles = getSampleStyleSheet()
    custom_style = ParagraphStyle(
        'CustomStyle',
        parent=styles['Normal'],
        fontName='DejaVu',
        fontSize=12,
    )

    pdf_file_path = "/kaggle/working/transcript.pdf"
    create_pdf(transcript_file_path, pdf_file_path)

    return "Аудиофайл успешно обработан. Транскрипция сохранена в transcript.txt, а PDF-файл в transcript.pdf"

bot.polling(none_stop=True, interval=0)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1762.07 ms
llama_print_timings:      sample time =      19.67 ms /    34 runs   (    0.58 ms per token,  1728.43 tokens per second)
llama_print_timings: prompt eval time =  248918.33 ms /  1164 tokens (  213.85 ms per token,     4.68 tokens per second)
llama_print_timings:        eval time =   10465.66 ms /    33 runs   (  317.14 ms per token,     3.15 tokens per second)
llama_print_timings:       total time =  260110.97 ms /  1197 tokens
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-li

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1762.07 ms
llama_print_timings:      sample time =      31.97 ms /    57 runs   (    0.56 ms per token,  1782.81 tokens per second)
llama_print_timings: prompt eval time =  149957.54 ms /   711 tokens (  210.91 ms per token,     4.74 tokens per second)
llama_print_timings:        eval time =   16811.83 ms /    56 runs   (  300.21 ms per token,     3.33 tokens per second)
llama_print_timings:       total time =  167395.19 ms /   767 tokens
